In [63]:
import numpy as np

In [19]:
# Lowering operator function
def lower_op(d):
    lower_op = np.zeros((d,d))
    for ii in range(0,d-1,1):
        lower_op[ii,ii+1] = np.sqrt(ii+1) 
        
    return lower_op

In [27]:
# Create operators for cavity and transmon
dc = 60
dq = 3

Ic = np.identity(dc)
Iq = np.identity(dq)

lowerc = lower_op(dc)
lowerq = lower_op(dq)

raisec = np.conjugate(np.transpose(lowerc))
raiseq = np.conjugate(np.transpose(lowerq))

In [33]:
# Static Hamiltonian parameters (note the units are GHz)

ωq = 5.1*2*np.pi
ωc = 7.2*2*np.pi
α = -0.35*2*np.pi
g = 0.2*2*np.pi

In [39]:
# Static Hamiltonian

H0 = ωc*np.kron(Iq,raisec@lowerc) + ωq*np.kron(raiseq@lowerq,Ic) + (α/2)*np.kron(raiseq@raiseq@lowerq@lowerq,Ic) + g*(np.kron(lowerq,raisec) + np.kron(raiseq,lowerc))


In [36]:
# Some physics to calculate the right drive frequency for our standard setup

Δ = ωq - ωc
χ01 = g**2/Δ
χ = (g**2/Δ/(Δ+α)*α)

In [58]:
# Pulse shape for the drive

def step_f(t,t_c):
    return 0.5 * (np.sign(t-t_c) + 1)

def gaussian(t,sig, ti, tf):
    return  np.exp(-(t - (tf+ti)/2)**2 / (2 * sig**2))

def gaussian_square(t, rise, Δt):
    return (step_f(t,rise) - step_f(t,Δt-rise)) + (step_f(t,0.0) - step_f(t,rise))*gaussian(t,rise/4,0.,2*rise) + (step_f(t,Δt-rise) - step_f(t,Δt))*gaussian(t+2*rise-Δt,rise/4,0.,2*rise)

In [65]:
# Drive parameters

Ω = 2*np.pi*1e-3
ωd = ωc - (χ01 - χ)
Δt0 = 350.0
rise0 = 8.0

In [59]:
# Drive operator and function

Hd = Ω*np.kron(Iq,raisec+lowerc)

def drivef(t):
    return 2*np.cos(ωd*t)*gaussian_square(t,rise0,Δt0)

In [55]:
# Dissipation

# Lindblad operators
L0 = np.kron(Iq,lowerc)
L1 = np.kron(lowerq,Ic)
L2 = np.kron(raiseq*lowerq,Ic)

# Rates
γ0 = 4*1e-3*2*np.pi
γ1 = 1/(350*1e3)
γ2 = 1/(230*1e3) - γ1/2

In [56]:
# Other parameters

# Initial state
rho_in = np.zeros((dq*dc,dq*dc))
rho_in[0,0] = 1.0

# Simulation time
t0 = 0.0
tf = 700.0

#### The rest of these cells define operators to measure the expectation value of

In [69]:
evs, evecs = np.linalg.eig(H0)

In [170]:
# Now this is where things get a bit tricky. We have to sort the eigenvectors so that the basis transformation 
# matrix respects the ordering of Hilbert spaces we imagine for the dressed states: 
# i.e. dressed transmon \otimes dressed cavity
# To do this we use that the hybridization is weak so we can sort using the bare projectors

# Projectors onto bare transmon states
Pg = np.kron(np.array([[1.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]]),Ic)
Pe = np.kron(np.array([[0.0,0.0,0.0],[0.0,1.0,0.0],[0.0,0.0,0.0]]),Ic)
Pf = np.kron(np.array([[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,1.0]]),Ic)

g_ol = []
e_ol = []
f_ol = []

for i in range(dc*dq):
    g_ol.append(np.trace(Pg@np.outer(evecs[:,i],evecs[:,i])))
    e_ol.append(np.trace(Pe@np.outer(evecs[:,i],evecs[:,i])))
    f_ol.append(np.trace(Pf@np.outer(evecs[:,i],evecs[:,i])))


# Sort by largest to smallest overlap
g_sort = np.argsort(g_ol).tolist()
g_sort.reverse()
e_sort = np.argsort(e_ol).tolist()
e_sort.reverse()
f_sort = np.argsort(f_ol).tolist()
f_sort.reverse()

# Take the largest overlaps
g_ind = g_sort[0:dc]
e_ind = e_sort[0:dc]
f_ind = f_sort[0:dc]

# Confirm no duplicate indices
temp = g_ind + e_ind + f_ind
print(len(temp) == len(set(temp)))

# Now since each of the individual lists are ordered by decreasing overlap with the transmon projector, they
# should be in reverse order of cavity overlap, but just to be safe let's order them. Upon inspection it turns
# out that they actually are not due to the finite truncation, so let's reorder them.

Nc = np.kron(Iq,raisec@lowerc)

c_ol_g = []
c_ol_e = []
c_ol_f = []
for j in range(dc):
    c_ol_g.append(np.trace(Nc@np.outer(evecs[:,g_ind[j]],evecs[:,g_ind[j]])))
    c_ol_e.append(np.trace(Nc@np.outer(evecs[:,e_ind[j]],evecs[:,e_ind[j]])))
    c_ol_f.append(np.trace(Nc@np.outer(evecs[:,f_ind[j]],evecs[:,f_ind[j]])))
    
# Sort by smallest to largest cavity overlap
gc_sort = np.argsort(c_ol_g).tolist()
ec_sort = np.argsort(c_ol_e).tolist()
fc_sort = np.argsort(c_ol_f).tolist()

g_ind_fullsort = [g_ind[k] for k in gc_sort]
e_ind_fullsort = [e_ind[k] for k in ec_sort]
f_ind_fullsort = [f_ind[k] for k in fc_sort]

# Confirm no duplicate indices again
temp2 = g_ind_fullsort + e_ind_fullsort + f_ind_fullsort
print(len(temp2) == len(set(temp2)))

# Finally we reorder the eigenvector matrix to create the change of basis matrix
V = np.zeros((dq*dc,dq*dc))

for n in temp2:
    V[:,n] = evecs[:,temp2[n]]
    
# For sanity check that V still does what it should and diagonalizes H0
tempM = np.conjugate(np.transpose(V))@H0@V
print(np.max(np.abs(tempM - np.diag(np.diagonal(tempM)))))

True
True
3.619461881965678e-11


In [171]:
# Transform dressed cavity lowering and raising operator back into lab frame
lowerc_dr = V@np.kron(Iq,lowerc)@np.conjugate(np.transpose(V))
raisec_dr = V@np.kron(Iq,raisec)@np.conjugate(np.transpose(V))

In [173]:
# Things to measure about the dressed cavity

# 1) Photon number operator
Nc_dr = raisec_dr@lowerc_dr

# 2) Cavity quadratures

X_dr = (lowerc_dr + raisec_dr)/np.sqrt(2)
Y_dr = -1j*(lowerc_dr - raisec_dr)/np.sqrt(2)

In [176]:
# Things to measure about the dressed transmon: state populations
# This creates projectors that sum over all the dressed eigenstates that correspond to a given dressed
# transmon state, as opposed to just picking the one with zero dressed cavity excitations

# G-state pop
g_proj = np.zeros((dq*dc,dq*dc))
for m in range(dc):
    g_proj = g_proj + np.outer(evecs[:,g_ind[j]],evecs[:,g_ind[j]])

# E-state pop
e_proj = np.zeros((dq*dc,dq*dc))
for m in range(dc):
    e_proj = e_proj + np.outer(evecs[:,e_ind[j]],evecs[:,e_ind[j]])

# F-state pop
f_proj = np.zeros((dq*dc,dq*dc))
for m in range(dc):
    f_proj = f_proj + np.outer(evecs[:,f_ind[j]],evecs[:,f_ind[j]])